In [2]:
import glob
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers,models
from sklearn.metrics import classification_report, confusion_matrix
import os 
import pickle

from sklearn import svm

In [3]:
def loadData(path):
    imgBGR2 = cv2.imread("data/personA/Test/personA_29.png")
    imgBGR2 = cv2.resize(imgBGR2,(300,300))
    imgGray2 = cv2.cvtColor(imgBGR2, cv2.COLOR_BGR2GRAY)
    return imgBGR2,imgGray2

In [4]:
def laodModels():
    cnnClassification = models.load_model('./part1Model')
    svms = pickle.load(open("part2.pkl", 'rb'))
    s = pickle.load(open("part2_2.pkl", 'rb'))
    return cnnClassification,svms,s


In [5]:
def normalization(features:np.ndarray, o):
    if o == 1:
        m = features.mean(axis=1)
        s = features.std(axis=1) 
        features = (features.T-m)/(s+1e-7)
        return features.T.flatten()
    if features.ndim > 1:
        features = features.flatten()
    m = features.mean()
    s = features.std()
    return (features-m)/(s+1e-7)

In [6]:
def featureExtractionM(img,o=0):
    features=0
    if o == 0: 
        hog = cv2.HOGDescriptor()
        img= cv2.resize(img, (64,128))
        features = hog.compute(img)
        features = normalization(features, o)

    elif o == 1:
        sift = cv2.SIFT_create()
        img= cv2.resize(img, (128,128))
        kp1, features = sift.detectAndCompute(img,None)
        if features.shape[0]<=64:
            features = np.concatenate([features,np.zeros((64-features.shape[0],128))], axis = 0)
        else:
            features = features[0:64,:]
        features = normalization(features, o)

    elif o == 2:
        img= cv2.resize(img, (64,32))
        features = cv2.Canny(img, 32, 128)
        features = normalization(features, o)
        
    elif o == 3:
        img= cv2.resize(img, (128,64))
        dst = cv2.GaussianBlur(img,(3,3),cv2.BORDER_DEFAULT,sigmaX=0.1,sigmaY=0.1)
        featuresX = cv2.Sobel(src=dst, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
        featuresY = cv2.Sobel(src=dst, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
        features =np.arctan(featuresY/(featuresX+1e-7))
        # sobelxy = cv2.Sobel(src=dst, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection
        features = normalization(features, o)        
        
    elif o == 4:
        img= cv2.resize(img, (128,64))
        dst = cv2.GaussianBlur(img,(3,3),cv2.BORDER_DEFAULT,sigmaX=0.1,sigmaY=0.1)
        featuresX = cv2.Sobel(src=dst, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
        featuresY = cv2.Sobel(src=dst, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
        features = featuresY + featuresX
        features = normalization(features, o) 
        
    elif o == 5:
        img = np.float32(img)
        features = cv2.cornerHarris(img,2,3,0.04)
        #result is dilated for marking the corners, not important
        features = cv2.dilate(features,None) 
        features = normalization(features, o)    
        features = features[features>0.01]  
        if features.shape[0]<=15000:
            features = np.concatenate([features,np.zeros((15000-features.shape[0]))], axis = 0)
        else:
            features = features[0:15000]
    return features


In [11]:
def detect(cnn,svms,svm,bgr,gray):
    c = np.argmax(cnn.predict_step(np.reshape(bgr/255,(1,300,300,3))))
    X_features = featureExtractionM(gray,o=0)
    names = ["personA","personB","personC","personD","personE"]
    svm = svms[names[c]]
    s = svm.predict(np.reshape(X_features,(1,-1)))
    return c, s[0]

In [12]:
bgr, gray =loadData("data/personA/Test/personA_29.png")

In [13]:
cnn,svms,s =laodModels()

In [14]:
detect(cnn,svms,svm,bgr,gray)

(0, 1)